In [1]:
import os
os.chdir("/home/udit/ipython/notebook/all/input")
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
np.random.seed(123)
from subprocess import check_output
import xgboost as xgb
import gc

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb

from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import KFold
from scipy.stats import skew, boxcox
from sklearn.preprocessing import StandardScaler
import itertools

shift = 200
COMB_FEATURE = 'cat80,cat87,cat57,cat12,cat79,cat10,cat7,cat89,cat2,cat72,' \
               'cat81,cat11,cat1,cat13,cat9,cat3,cat16,cat90,cat23,cat36,' \
               'cat73,cat103,cat40,cat28,cat111,cat6,cat76,cat50,cat5,' \
               'cat4,cat14,cat38,cat24,cat82,cat25'.split(',')

def encode(charcode):
    r = 0
    ln = len(str(charcode))
    for i in range(ln):
        r += (ord(str(charcode)[i]) - ord('A') + 1) * 26 ** (ln - i - 1)
    return r

In [3]:
fair_constant = 2
def fair_obj(preds, dtrain):
    labels = dtrain.get_label()
    x = (preds - labels)
    den = abs(x) + fair_constant
    grad = fair_constant * x / (den)
    hess = fair_constant * fair_constant / (den * den)
    return grad, hess

def xg_eval_mae(yhat, dtrain):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y)-shift,
                                      np.exp(yhat)-shift)
def mungeskewed(train, test, numeric_feats):
    ntrain = train.shape[0]
    test['loss'] = 0
    train_test = pd.concat((train, test)).reset_index(drop=True)
    skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna()))
    skewed_feats = skewed_feats[skewed_feats > 0.25]
    skewed_feats = skewed_feats.index

    for feats in skewed_feats:
        train_test[feats] = train_test[feats] + 1
        train_test[feats], lam = boxcox(train_test[feats])
    return train_test, ntrain

In [4]:
print('\nStarted')
directory = '../input/'
#train = pd.read_csv(directory + 'train.csv.zip')
#test = pd.read_csv(directory + 'test.csv.zip')

num_folds = 2 #should be larger, but kaggle scirpts has run time 

def MAE(y,dtrain):
    answer = dtrain# dtrain.get_label()
    answer = np.array(answer)
    prediction = np.array(y)
    error = np.exp(prediction) -np.exp(answer)
    error = np.mean((error**2)**.5)
    return 'mcc error',error
    
def MAE2(y,dtrain):
    answer = dtrain.loss2
    answer = np.array(answer)
    prediction = np.array(y)
    error = prediction - answer
    error = np.mean((error**2)**.5)
    return 'mcc error',error


        
parameters =[]
for i in (6,12):
    for j in (60,):
            for l in (1,2):
                depth = i
                min_child_weight = j
                gamma=l
                parameters += [[depth,min_child_weight,gamma],]


Started


In [43]:
train=pd.read_pickle("all_train_processed.pkl")
test=pd.read_pickle("all_test_processed.pkl")

In [44]:
shift = 200
ids = pd.read_csv('../input/test.csv.zip')['id']

In [45]:
train['logloss'] =np.log(train['loss']+shift) 
predictors = list(test.columns[2:len(train.columns)])
target='logloss'
result={}

In [46]:
train=train.astype(float)
test=test.astype(float)

In [47]:
import os
from pylightgbm.models import GBMRegressor
os.environ['LIGHTGBM_EXEC'] = "/home/udit/LightGBM/lightgbm"
#https://github.com/ArdalanM/pyLightGBM

In [48]:
## train 4 models with different paremeters ###
n_folds = 10
cv_sum = 0
early_stopping = 100
#xgtest=xgb.DMatrix(test[predictors].values,missing=np.NAN,feature_names=predictors)
for i,j,l in parameters:
    depth,min_child_weight,gamma=i,j,l
    result[(depth,min_child_weight,gamma)]=[]
    ### name of prediction ###
    name = 'feature_L2_%s_%s_%s_%s' %(str(depth), str(min_child_weight), str(gamma),str(num_folds))
    train  [name]=0
    test[name]=0
    for fold in range(0,n_folds):
        print ('\ntraining  parameters', i,j,l,',fold',fold)
        gc.collect() #to clear ram of garbage
        train_i = [x for x in train.index if x%num_folds != fold]
        cv_i = [x for x in train.index if x%num_folds == fold]
        dtrain= train.iloc[train_i]
        dcv = train.iloc[cv_i]
        model = GBMRegressor(
            num_threads=-1,
            boosting_type='gbdt',
            learning_rate = 0.3,
            num_iterations = 10, 
            tree_learner="serial",
            #max_depth=8,
            #lambda_l2=gamma,
            verbose = False, 
            early_stopping_round = 10,
            feature_fraction = 0.7,
            bagging_fraction = 0.7,
            min_sum_hessian_in_leaf=min_child_weight
           )        
        model.fit(
        dtrain[predictors].values, 
        dtrain[target].values, 
        test_data = [(dcv[predictors].values, dcv[target].values )]
        )
        print ('train_result',MAE(model.predict(dcv[predictors].values),dcv[target].values))
        train.set_value(cv_i,name,np.exp(model.predict(dcv[predictors].values))-200)
        test.set_value(test.index,name,test[name]+((np.exp(model.predict(test[predictors].values))-200)/num_folds))
        del model
        gc.collect()

('\ntraining  parameters', 6, 60, 1, ',fold', 0)
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
('train_result', ('mcc error', 737.77526500503382))
None
None

IOError: [Errno 2] No such file or directory: '/tmp/tmp9Rl_8i/LightGBM_model.txt'

In [53]:
model.fit(
dtrain[predictors].values, 
dtrain[target].values, 
 a
test_data = [(dcv[predictors].values, 
              dcv[target].values )],
    early_stopping_rounds = 15,
    verbose = False,
)


2

In [50]:
print ('train_result',MAE(model.predict(dcv[predictors].values),dcv[target].values))
train.set_value(cv_i,name,np.exp(model.predict(dcv[predictors].values))-200)
test.set_value(test.index,name,test[name]+((np.exp(model.predict(test[predictors].values))-200)/num_folds))
del model

TypeError: expected a character buffer object

In [79]:
#### NOW THE MCMC PART to find individal weights for ensemble####
features = [x for x in  train.keys() if 'feature' in x]
print ('features are these:', features)
num=len(features)
#intialize weights
weight = np.array([1.0/num,]*num)

('features are these:', ['feature_L2_6_60_1_2', 'feature_L2_6_60_2_2', 'feature_L2_12_60_1_2', 'feature_L2_12_60_2_2'])


In [83]:
# This is to define variables to be used later
train['pred_new']=0
train['pred_old']=0
counter = 0
n=1000 ###MCMC steps
result={}
train['loss2']=np.exp(train[target])-200

In [86]:
for i in range(0,len(features)):
    train['pred_new'] += train[features[i]]*weight[i]
    print ('feature:',features[i],',MAE=',MAE2(train[features[i]],train))
print ('combined all features',',MAE=', MAE2(train.pred_new,train))
train['pred_old']=train['pred_new']

('feature:', 'feature_L2_6_60_1_2', ',MAE=', ('mcc error', 72.489651185673395))
('feature:', 'feature_L2_6_60_2_2', ',MAE=', ('mcc error', 87.025322535962516))
('feature:', 'feature_L2_12_60_1_2', ',MAE=', ('mcc error', 74.404400722993998))
('feature:', 'feature_L2_12_60_2_2', ',MAE=', ('mcc error', 88.477803252070913))
('combined all features', ',MAE=', ('mcc error', 6021.4969435252424))


In [87]:
#### MCMC  #### 
### MCMC algo for dummies 
### 1. Get initialize ensemble weights
### 2. Generate new weights 
### 3. if MAE is lower, accept new weights immediately , or else accept new weights with probability of np.exp(-diff/.3)
### 4. repeat 2-3
for i in range(0,n):
    new_weights = weight+ np.array([0.005,]*num)*np.random.normal(loc=0.0, scale=1.0, size=num)
    new_weights[new_weights < 0.01]=0.01
    train['pred_new']=0
    for ii in range(0,len(features)):
         train['pred_new'] += train[features[ii]]*new_weights[ii]
    diff = MAE2(train.pred_new,train)[1] - MAE2(train.pred_old,train)[1]
    prob = min(1,np.exp(-diff/.3))
    random_prob = np.random.rand()
    if random_prob < prob:
        weight= new_weights
        train['pred_old']=train['pred_new']
        result[i] = (MAE2(train.pred_new,train)[1] ,MAE2(train.pred_old,train)[1],prob,random_prob ,weight)
        #print (MAE2(train.pred_new,train)[1] ,MAE2(train.pred_old,train)[1],prob,random_prob),
        counter +=1

In [88]:
print (counter *1.0 / n, 'Acceptance Ratio') #keep this [0.4,0.6] for best results
print ('best result MAE', sorted([result[i] for i in result])[0:1][0])

weight=sorted([result[i] for i in result])[0:1][-1]
train['pred_new']=0
for i in range(0,len(features)):
    train['pred_new'] += train[features[i]]*weight[i]
print ('combined all features plus MCMC weights:',',MAE=', MAE2(train.pred_new,train))

print ('weights:', weight[-1])
### notice the weights do not necessarily sum to 1 ###

(0.231, 'Acceptance Ratio')
('best result MAE', (78.357179563348723, 78.357179563348723, 1, 0.061771554343897694, array([ 0.27428704,  0.22573817,  0.2713866 ,  0.23074799])))
('combined all features plus MCMC weights:', ',MAE=', ('mcc error', 473342.92499919713))
('weights:', array([ 0.27428704,  0.22573817,  0.2713866 ,  0.23074799]))


In [95]:
test.loc[:,'loss']=0
for i in range(0,len(features)):
    test.loc[:,'loss'] += test[features[i]]*weights[i]

In [100]:
test[features].head()

,feature_L2_6_60_1_2,feature_L2_6_60_2_2,feature_L2_12_60_1_2,feature_L2_12_60_2_2
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [94]:
weights[0]

0.27428703767575863